# 1. 语义搜索引擎搭建
- **项目地址：https://python.langchain.com/docs/tutorials/retrievers/**

**本项目会使你熟悉LangChain的文档加载器、嵌入和向量存储抽象。这里我们将构建一个搜索引擎基于一个PDF文档，这将使我们能够检索PDF中与输入查询相似的段落。这个项目着重以下几个概念：**
- 文档及文档加载器；
- 文本分割器；
- 嵌入；
- 向量存储和提取器；

# 2. 环境设置
- Jupyter notebook or Jupyterlab

- 安装包
pip install langchain-community pypdf

# 3. LangSmith设置
- 用户过程监控，方便debug

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


# 4. 文档及文档加载器
- LangChain 可以将文本进行抽象，用以表示一段文本及其相关的元数据。 一般有三个属性：
    - page_cotent:代表内容的字符串
    - metadata:一个包含任意元数据的字典。可以用来捕捉文档相关的信息。
    - id：（选择项）文档的字符串标识符
- 单一文档对象-Document通常表示一些大的文档

In [6]:
# 从langchain_core来载入Document类，langchain_core在安装langchain-community时已安装好
from langchain_core.documents import Document
# 构建documents列表，里面包含多个document对象
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness",
        metadata = {"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammal-pets-doc"},
    ),
]
    # 单一Document对象，包含以上所说的属性


## 4.1.加载文档
让我们加载一个PDF文件到一组Document对象里。同一文件夹下有一个PDF样例，2023年的Nike文件，数据来自Langchain官方文档。接下来我们可以访问LangChain文档来查询可用的PDF文档加载器。我们这里选择PyPDFLoader。

In [7]:
# 载入PDFLoader
from langchain_community.document_loaders import PyPDFLoader
# 初始化了一个PyPDFLoader对象，用于加载指定路径的PDF文件
file_path = "./example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)
# 从指定的 PDF 文件中提取文本内容，并将其结构化为 LangChain 可用的格式，存储在变量 docs 中。
docs = loader.load()
# 返回列表的长度
print(len(docs))

107


- PyPDFLoader 会对PDF文件的每一页都构建一个Document对象，这样我们能轻易拿到该页字符串的数据以及包含了文件名和页数的元数据

In [9]:
# 打印PDF第一页的前200个字符并换行
print(f"{docs[0].page_content[:200]}\n")
# 打印第一页的元数据并显示为字典的形式
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


# 5.文本分割
- 对于信息检索以及为了回答下游的问题，一页PDF是不够的。我们的目标是检索Document对象以回答一个输入的问题，进一步分割PDF文件能确保检索到的相关内容的含义不被上下文给洗掉。
- 我们可以使用文本分割器来达成这一目的。这里我们将使用一个基于字符区分的文本分割器。我们将把我们的文件拆分成1000个字符的块，其中有200个字符块会重叠。重叠的部分会帮助减少相关概念陈述的拆分，导致意义不全。我们使用RecursiveCharacterTextSplitter这个文本分割器，它将使用新一行等分割符来递归地拆分文档，直到每个块都有合适的大小。对于一般的用例都推荐这个文本分割器。
- 我们使用 add_start_index=True 以便将字符索引保存为属性为“start_index”的元数据，这个字符索引是分割后的Document对象在原文件中的字符位置。

In [11]:
# langchain_text_splitters 是 langchain的一部分，langchain_community安装时会安装langchain，无需单独安装
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 初始化文本分割器，设置好相应的参数，后续分割时需要用到
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, #每个文本块的最大长度
    chunk_overlap=200, #相邻文本块之间的重叠长度，为了避免丢失上下文。
    add_start_index=True #在分割后的块中记录该块在原始文本中的起始位置
)

# 开始分割文档。split_documents会遍历docs中的每个文档，对每个文档的文本内容进行地柜分割。
all_splits = text_splitter.split_documents(docs)

# Document对象列表长度
len(all_splits)

516

# 6.嵌入
- 将document对象进行向量化处理
- 向量搜索是一个常用的方式来存储以及搜索非结构化的数据。主要目的是存储和文本相关联的数字向量。给定一个问题，我可以将其作为一个向量嵌入，这个向量会和文本转化后的向量处在同一向量纬度，然后使用向量相似性的方法去识别相关的文本。
- Langchain支持多种供应商的嵌入模型，这些模型规定了文本如何被转化成数字向量。
- **这里我们使用HuggingFace来做处理**
    - 为什么选择Hugging Face？
        - 完全免费
        - 丰富的预训练模型
        - 文档和社区强大
        - 与 LangChain 无缝集成
        - 适合入门和进阶

- 安装Hugging Face库  
    - pip install -qU langchanin-huggingface
- 代码解释
    - q（quiet）：静默安装，不显示进度条和冗长的日志信息，只在安装失败时显示错误信息。
    - U（upgrade）：如果已经安装了该库，则升级到最新版本。


In [1]:
# 它的作用是利用 Hugging Face 预训练模型将文本转换为向量，以便在 LangChain 框架中用于检索增强生成（RAG）等任务。
from langchain_huggingface import HuggingFaceEmbeddings


# 加载 Hugging Face 的 sentence-transformers 预训练模型
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


- HuggingFaceEmbeddings：LangChain 提供的一个封装类，专门用于调用 Hugging Face 的文本嵌入模型。
- model_name="sentence-transformers/all-mpnet-base-v2"：指定使用 sentence-transformers 旗下的 all-mpnet-base-v2 模型，它是一个性能较强的句向量（sentence embeddings）模型，常用于文本相似性计算、语义搜索等任务。

In [19]:
# 先拿 all_splits[0]/[1] 的 page_content部分，然后用 HuggingFaceEmbeddings.embed_query() 对文本进行向量化。
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

# 这行代码的作用是断言 vector_1 和 vector_2 的长度相等，如果不相等，就会触发AssertionError，终止程序执行。
# 确保 Hugging Face 的 embedding 模型输出维度一致。
# 防止因向量维度不匹配导致后续计算错误。
assert len(vector_1) == len(vector_2)

# 打印vector1的长度并打印vector_1的1-10的向量内容
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.04747236520051956, 0.021675819531083107, -0.009018069133162498, 0.005356764420866966, 0.02555764466524124, -0.010230285115540028, -0.008413959294557571, 0.03930392488837242, 0.021570494398474693, -0.02409539557993412]


- **配置好了文本向量化生成模型之后，我们下一步可以把他们储存到一个特殊的数据结构中，这个数据结构支持高效的相似性查询。**

# 7.向量存储
- LangChain向量存储对象包含了很多添加文本以及Document对象到向量数据库中的方法，并可以使用多种相似性度量来进行检索储存好的向量。我们经常使用嵌入模型来初始化这些向量，这些模型决定了如何将文本数据转换成数字向量。
- LangCahin集成了一套不同的向量存储技术。一些向量数据库有特定的服务商提供，一般是云服务，并需要特定的身份验证才能进行使用，但一些例如Postgres可以在独立的基础设施上运行，可以通过第三方服务或者本地部署，还有一些轻量化的向量数据库可以使用内存来运行，支持低负载任务。这里我们使用Faiss来进行向量存储。

**什么是FAISS**
- FAISS（Facebook AI Similarity Search）是由 Meta（原 Facebook）AI 研究团队 开发的一款高效相似性搜索库，主要用于大规模向量数据的快速搜索和检索。它特别适用于高维嵌入向量的最近邻搜索（Nearest Neighbor Search, NNS），在机器学习、自然语言处理（NLP）、推荐系统等领域应用广泛。
- FAISS 的作用
    - ✅ 快速最近邻搜索（Approximate Nearest Neighbor, ANN）：在海量向量数据中找到最相似的向量（如文本、图片、音频等）。
    - ✅ 支持高维向量检索：适用于 NLP、计算机视觉等领域的嵌入向量搜索（如 Sentence Transformers、CLIP 等）。
    - ✅ 内存 & 磁盘优化：提供不同的索引方式，支持在 CPU 或 GPU 上高效运行，减少内存占用，提升搜索速度。
    - ✅ 支持索引压缩（Index Compression）：可以用更少的存储空间处理大规模数据，提高查询效率。
    - ✅ 与机器学习框架兼容：可与 PyTorch、TensorFlow、Hugging Face 等 框架结合使用。

- **安装faiss-cpu依赖**
- pip install faiss-cpu
- langchain-community 封装的 FAISS 类只是一个包装器，它提供了一组接口，使得你可以在 LangChain 中更方便地使用 FAISS 的功能。但是，实际的 FAISS 实现代码并不在 langchain-community 中，而是由 Facebook 开源的 FAISS 库提供。因此，你需要安装底层依赖（即 faiss-cpu 或 faiss-gpu）来支持真正的向量索引和相似性搜索功能。
- faiss-cpu vs faiss-gpu
    - faiss-cpu：仅基于 CPU 运行，没有 GPU 加速。适用于没有 GPU 或不需要 GPU 加速的场景。
    - faiss-gpu：支持 GPU 加速，能够显著提高大规模向量搜索的性能。需要配置 CUDA 环境和兼容的 GPU。


In [26]:
# langchain_community有封装FAISS的类，可以直接进行使用。
from langchain_community.vectorstores import FAISS
# 调用初始化好的预训练模型，对分割好的Document文件进行向量化并存储
# LangChain在上述类中进行了逻辑处理，使得Hugging face和FAISS的连接可以用下列方式实现
vector_store = FAISS.from_documents(all_splits,embeddings)

- 当我们完成了Document对象的向量存储的实例化，我们就可以进行查询了。向量数据库包含了多种查询方式：
    - 同步以及异步；
    - 字符串查询或者向量查询；
    - 是否返回相似性分数；
    - 相似性和最大边际相关性（平衡相似性和检索结果的多样性）
- 输出结果也是Document对象

# 8.向量数据库使用
- 嵌入通常将文本表示为一个浓缩的向量，这样相似的文本就能在几何意义上靠近彼此。这使得我们能够通过问题检索到相关信息，而不需要使用任何文档中的特定术语知识。

In [31]:
# 基于相似性返回一个查询的文档结果
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 

In [32]:
# Async query
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

- 同、异步查询区别
    - 同步查询：代码等待查询完成，适用于小规模查询，逻辑简单但可能影响性能。
    - 异步查询：代码不会阻塞，可以并行处理多个查询，提高效率，适用于大规模检索场景。

### Return scores

In [33]:
# 不同的服务商会采取不同的打分方法
# 这里的分数是一个距离度量，与相似性的变化呈反比

# results 是一个列表，包含多个元组，每个元组包含一个Document对象一个相似度分数Score
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score:{score}\n")
print(doc)

Score:0.3725222945213318

page_content='Table of Contents
YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022 2021
REVENUES
North America $ 21,608 $ 18,353 $ 17,179 
Europe, Middle East & Africa 13,418 12,479 11,456 
Greater China 7,248 7,547 8,290 
Asia Pacific & Latin America 6,431 5,955 5,343 
Global Brand Divisions 58 102 25 
Total NIKE Brand 48,763 44,436 42,293 
Converse 2,427 2,346 2,205 
Corporate 27 (72) 40 
TOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710 $ 44,538 
EARNINGS BEFORE INTEREST AND TAXES
North America $ 5,454 $ 5,114 $ 5,089 
Europe, Middle East & Africa 3,531 3,293 2,435 
Greater China 2,283 2,365 3,243 
Asia Pacific & Latin America 1,932 1,896 1,530 
Global Brand Divisions (4,841) (4,262) (3,656)
Converse 676 669 543 
Corporate (2,840) (2,219) (2,261)
Interest expense (income), net (6) 205 262 
TOTAL NIKE, INC. INCOME BEFORE INCOME TAXES $ 6,201 $ 6,651 $ 6,661 
ADDITIONS TO PROPERTY, PLANT AND EQUIPMENT
North America $ 283 $ 146 $ 98 
Europe, Middle East & Africa 215

### 根据嵌入查询的相似性返回文档。
- 系统会将输入的查询（query）转换为向量（embedding），然后在存储的向量数据库中查找与该查询最相似的文档，并返回这些文档。

In [34]:

embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

# 9. 检索器
- 大意： @chain 让 retriever 变成 Runnable，支持 .batch()、.invoke() 等方法。
- LangChain的VectorStore对象不会继承Runnable类。LangChain检索器属于Runnables类，所以他有一套标准的方法（例如同步和异步调用及批量操作）。尽管我们能够从向量数据库创建检索器，但检索器也可以与非向量存储的数据源进行交互，例如外部API。
- 我们可以自己创建一个简单的版本，而不必继承 Retriever 类。如果我们选择希望使用的文档检索方法，就可以很容易地创建一个 Runnable。下面我们将基于 similarity_search 方法构建一个。

- 什么是Runnables类？   

    - 在 LangChain 中，“Runnables” 是一种设计模式或接口概念，用于表示那些可以统一调用的组件。也就是说，实现了 Runnables 接口的组件会提供一套标准的方法，比如：
    
        - 同步调用（例如 run 方法）：调用后等待结果返回；
        - 异步调用（例如 arun 方法）：调用后可以并发执行其他任务；
        - 批量操作：一次处理多个输入。  

    - 这种设计的好处在于，不管具体实现是什么，只要遵循了 Runnables 接口，就可以以相同的方式被调用和集成。例如，LangChain 的检索器（Retrievers）通常实现了 Runnables 接口，这样你就可以用统一的方法来触发它们去查询相关文档，而不用关心内部是如何完成查询的。

    - 相反，VectorStore 对象主要负责存储和管理向量数据，它并不需要提供执行（run）功能，所以它不会实现 Runnables 接口。

    - 总之，Runnables 类就是指那些可以“运行”或“调用”的组件，它们实现了一套标准接口，以便于在框架中统一使用。

In [36]:
# 这是 Python 内置的类型提示库，用来引入泛型 List 类型。
# 它允许你在函数签名中指定返回值或变量的类型（例如 List[Document] 表示文档列表）。
from typing import List 
# Document：这是 LangChain 用来表示文档的类，通常包含文本内容和相关元数据。
from langchain_core.documents import Document
# 这是一个装饰器，用于将普通函数包装成一个“Runnable”，即一个具有统一接口（如同步、异步、批处理等调用方式）的组件。
from langchain_core.runnables import chain 

# @chain装饰器：将 retriever 函数包装成一个 Runnable 对象，赋予它标准化的调用接口。
@chain
# 函数签名：表示该函数接受一个字符串类型的查询，并返回一个 Document 对象组成的列表。
def retriever(query: str) -> List[Document]:
    # 对传入的查询进行相似性搜索，返回最相关的（k=1 表示只返回1个结果）。
    return vector_store.similarity_search(query, k=1)

# 这是 Runnable 对象（由 @chain 装饰后生成的）的批量调用接口。
# 一次性处理一批查询，而不是每次单独调用。
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='c6ee2f29-c7f2-4adb-b7ab-6c3020c88efa', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and

- 向量数据库还使用了一种叫as_retriever的方法来生成检索器，具体来说是VectorStoreRetriever。这些检索器包含了特定的search_type 和 search_kwargs 属性， 这些属性能够识别底层向量存储需要调用的方法，并知道如何参数化他们。例如，我们可以将以上的代码思路这样实现：

In [37]:
# as_retriever 方法会将 vector_store 转换为一个 Retriever 对象，允许它像一个检索器一样工作。
retriever = vector_store.as_retriever(
    # 指定检索类型为相似度检索（similarity search）。
    search_type="similarity",
    # search_kwargs 是一个字典，用于传递检索的额外参数。
    # 这里的 k=1 表示返回最相似的 1 个结果（即仅返回最匹配的一个文档）。
    search_kwargs={"k":1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ]

)

[[Document(id='c6ee2f29-c7f2-4adb-b7ab-6c3020c88efa', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and

- VectorStoreRetriever 默认支持相似性搜索，最大边际相关性mmr以及“similarity_score_threshold”.我们可以使用后者通过相似度分数对检索器输出的文档进行筛选。
- 检索器可以轻易的被嵌入到更复杂的应用中，例如RAG 应用。了解更多应用构建知识，可以查看RAG教学文档：https://python.langchain.com/docs/tutorials/rag/